<a href="https://colab.research.google.com/github/Aman17Javed/Supervised-Fine-Tuning-unsloth-llama-3.2-3b-bnb-4bit/blob/main/SFT_Llama_3_2_3B_bnb_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/

In [2]:
!pip install --upgrade transformers accelerate safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0


In [3]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import torch


In [5]:
max_seq_length=2048

In [6]:
dtype=None
load_in_4bit=True

In [7]:
from huggingface_hub import login

In [8]:
from google.colab import userdata
hf_token=userdata.get('hugging_face_key')
login(hf_token)

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=hf_token,
)

==((====))==  Unsloth 2025.8.5: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [10]:
prompt_style = """Below is an instruction that describes a task, paired with a question.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a detailed answer.

### Instruction:
You are a helpful AI assistant with advanced knowledge in multiple fields.
Please answer the following question.

### Question:
{}

### Response:
<think>{}
"""


In [11]:
model = FastLanguageModel.get_peft_model(

model,

r=16,

target_modules=[

"q_proj",

"k_proj",

"v_proj",

"o_proj",

"gate_proj",

"up_proj",

"down_proj",

],

lora_alpha=16,

lora_dropout=0,

bias="none",

use_gradient_checkpointing="unsloth",

random_state=9001,

use_rslora=False,

loftq_config=None,
)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [12]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Define how each training sample should look
train_prompt_style = """Instruction:
{instruction}

Context:
{context}

Response:
{response}
"""

def formatting_prompts_func(examples):
    # Get columns with safe defaults
    inputs = examples.get("instruction", [""] * len(examples["response"]))
    contexts = examples.get("context", [""] * len(examples["response"]))  # Dolly has context column
    outputs = examples.get("response", [""] * len(examples["response"]))

    texts = []
    for instr, ctx, resp in zip(inputs, contexts, outputs):
        # Fill in prompt
        text = train_prompt_style.format(
            instruction=instr.strip(),
            context=ctx.strip(),
            response=resp.strip()
        ) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}


In [13]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k")
dataset = dataset.map(formatting_prompts_func, batched=True)

print(dataset["train"]["text"][0])  # Show first formatted prompt


README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Instruction:
When did Virgin Australia start operating?

Context:
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.

Response:
Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.
<|end_of_text|>


In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# ✅ Sample only 500 rows for speed
train_dataset = dataset["train"].shuffle(seed=42).select(range(500))

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",  # Make sure your formatting function created this field
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,  # ⏳ ~3-5 min training
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    )
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
trainer_stats=trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.110800
20,1.986000
30,1.896100
40,1.835100
50,1.847100
60,1.932600
70,1.801300
80,1.677700
90,1.737400
100,1.706500


In [16]:
# Save model + tokenizer in Hugging Face format
save_dir = "outputs"  # folder to store files

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('outputs/tokenizer_config.json',
 'outputs/special_tokens_map.json',
 'outputs/tokenizer.json')

In [21]:

from transformers import AutoTokenizer
from unsloth import FastLanguageModel

model_path = "unsloth/Llama-3.2-3B-bnb-4bit"
max_seq_length = 512
dtype = "float16"
load_in_4bit = True

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model
model, _ = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

# Enable inference optimizations
FastLanguageModel.for_inference(model)

# Define the prompt
prompt = """Explain supervised fine-tuning in simple words. What is the difference between fine-tuning and training from scratch? What are the benefits of fine-tuning? How do I fine-tune a model? How do I fine-tune a model using Hugging Face Transformers?"""

# Generate text with tuned settings
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,  # Increased for longer response
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode and print output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


==((====))==  Unsloth 2025.8.5: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Explain supervised fine-tuning in simple words. What is the difference between fine-tuning and training from scratch? What are the benefits of fine-tuning? How do I fine-tune a model? How do I fine-tune a model using Hugging Face Transformers? What are the benefits of fine-tuning? What are the differences between fine-tuning and training from scratch?
Supervised fine-tuning is a process of fine-tuning a pre-trained model to a specific task, which is typically done on a dataset of labeled data. The pre-trained model is typically a large la

In [28]:
new_model_online = "amanjaved421/SFT_Llama-3.2-3B-bnb-4bit"
new_model_local = "SFT_Llama-3.2-3B-bnb-4bit"

model.save_pretrained(new_model_local)     # Save model weights
tokenizer.save_pretrained(new_model_local) # Save tokenizer


('SFT_Llama-3.2-3B-bnb-4bit/tokenizer_config.json',
 'SFT_Llama-3.2-3B-bnb-4bit/special_tokens_map.json',
 'SFT_Llama-3.2-3B-bnb-4bit/tokenizer.json')

In [29]:
model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

README.md:   0%|          | 0.00/583 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...a-3.2-3B-bnb-4bit/model.safetensors:   1%|          | 16.8MB / 2.24GB            

Saved model to https://huggingface.co/amanjaved421/SFT_Llama-3.2-3B-bnb-4bit


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...lama-3.2-3B-bnb-4bit/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/amanjaved421/SFT_Llama-3.2-3B-bnb-4bit/commit/eb6151c843fb3ceda349d6dab7f760273dea3e6c', commit_message='Upload tokenizer', commit_description='', oid='eb6151c843fb3ceda349d6dab7f760273dea3e6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/amanjaved421/SFT_Llama-3.2-3B-bnb-4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='amanjaved421/SFT_Llama-3.2-3B-bnb-4bit'), pr_revision=None, pr_num=None)